In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving EMAILS_FINAL.csv to EMAILS_FINAL.csv
User uploaded file "EMAILS_FINAL.csv" with length 8937578 bytes


In [37]:
import chardet
import pandas as pd
file='EMAILS_FINAL.csv'
with open(file,'rb') as rawdata:
  result=chardet.detect(rawdata.read(100000))
result
df=pd.read_csv(file,encoding='ISO-8859-1',delimiter=',',quotechar='"')
df=df.dropna()
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


# Import the libraries :

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [38]:
df.shape

(5728, 2)

In [39]:
df.columns

Index(['text', 'spam'], dtype='object')

In [40]:
df.drop_duplicates(inplace=True)
print(df.shape)

(5695, 2)


In [41]:
print(df.isnull().sum())

text    0
spam    0
dtype: int64


In [42]:
# download the stopwords package
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [43]:
def process(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)

    clean = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    return clean
# to show the tokenization
df['text'].head().apply(process)

0    [Subject, naturally, irresistible, corporate, ...
1    [Subject, stock, trading, gunslinger, fanny, m...
2    [Subject, unbelievable, new, homes, made, easy...
3    [Subject, 4, color, printing, special, request...
4    [Subject, money, get, software, cds, software,...
Name: text, dtype: object

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
message = CountVectorizer(analyzer=process).fit_transform(df['text'].values.astype('U'))

In [45]:
#split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(message, df['spam'], test_size=0.20, random_state=0)
# To see the shape of the data
print(message.shape)

(5695, 37105)


In [46]:
# create and train the Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(xtrain, ytrain)

In [47]:
print(classifier.predict(xtrain))
print(ytrain.values)

['0'
 ' its termination would not  have such a phenomenal impact on the power situation .  however '
 '0' ... '0' '0' '0']
['0' '0' '0' ... '0' '0' '0']


In [48]:
# Evaluating the model on the training data set
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(xtrain)
print(classification_report(ytrain, pred))
print()
print("Confusion Matrix: \n", confusion_matrix(ytrain, pred))
print("Accuracy: \n", accuracy_score(ytrain, pred))

                                                                                              precision    recall  f1-score   support

 its termination would not  have such a phenomenal impact on the power situation .  however        0.00      0.00      0.00         1
                                                                           mr suresh prabhu        0.00      0.00      0.00         1
                                                                                           0       1.00      0.99      1.00      3455
                                                                                           1       0.99      1.00      0.99      1099

                                                                                    accuracy                           0.99      4556
                                                                                   macro avg       0.50      0.50      0.50      4556
                                                            

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [49]:
#print the predictions
print(classifier.predict(xtest))
#print the actual values
print(ytest.values)

['1' '0' '0' ... '0' '0' '0']
['1' '0' '0' ... '0' '0' '0']


In [51]:
#Now let’s evaluate the model on the test data set :

# Evaluating the model on the training data set
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(xtest)
print(classification_report(ytest, pred))
print()
print("Confusion Matrix: \n", confusion_matrix(ytest, pred))
print("Accuracy: \n", accuracy_score(ytest, pred))

                                                                                              precision    recall  f1-score   support

 its termination would not  have such a phenomenal impact on the power situation .  however        0.00      0.00      0.00         0
                                                                           mr suresh prabhu        0.00      0.00      0.00         0
                                                                                           0       1.00      0.99      0.99       870
                                                                                           1       0.98      0.97      0.97       269

                                                                                    accuracy                           0.98      1139
                                                                                   macro avg       0.49      0.49      0.49      1139
                                                            

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
